# Importing libaries

In [11]:
import requests
import json
from google.colab import files
import pandas as pd
from geopy.geocoders import Nominatim

## Function get_stores

Get a location address string and returns the stores dataframe which Rappi attends in that location

In [12]:
def get_stores(address):
  # Getting location data with geopy Nominating
  geolocator = Nominatim(user_agent="Rappi")
  location = geolocator.geocode(address)
  
  headers = {
      'authority': 'services.rappi.com.br',
      'pragma': 'no-cache',
      'cache-control': 'no-cache',
      'deviceid': 'fcabdfa7-054d-4fae-9c95-d30bd4719af7',
      'language': 'pt',
      'accept-language': 'pt-BR',
      'sec-ch-ua-mobile': '?0',
      'content-type': 'application/json',
      'accept': 'application/json, text/plain, */*',
      'app-version': 'web_4.20.6',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
      'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"',
      'origin': 'https://www.rappi.com.br',
      'sec-fetch-site': 'same-site',
      'sec-fetch-mode': 'cors',
      'sec-fetch-dest': 'empty',
      'referer': 'https://www.rappi.com.br/',
  }

  # Setting the data with latitude and longitude of the address passed in the function
  data = '{"state":{"lat": "' + str(location.latitude) + '","lng":"' + str(location.longitude) + '","parent_store_type":"market"},"limit":100,"offset":0,"context":"cpgs_landing"}'
  # Get response from rappi
  response = requests.post('https://services.rappi.com.br/api/dynamic/context/content', headers=headers, data=data)
  # Read json data from the response
  json_data = json.loads(response.text)
  # Stores dataframe from the response
  stores = pd.json_normalize(json_data['data']['context_info']['stores'])
  
  return stores

# Function rappi

With a list of store ids and a keyword, scrapes the products found in each store

In [13]:
def rappi(stores,keyword):
  headers = {
      'authority': 'services.rappi.com.br',
      'accept': 'application/json, text/plain, */*',
      'accept-language': 'pt-BR',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36',
      'content-type': 'application/json',
      'origin': 'https://www.rappi.com.br',
      'sec-fetch-site': 'same-site',
      'sec-fetch-mode': 'cors',
      'sec-fetch-dest': 'empty',
      'referer': 'https://www.rappi.com.br/',
  }
  
  # Setting the data for the store list and keywrd passed in the function
  data = '{"store_ids":' + str(stores.tolist()) + ', "keyword":' + keyword + '}'
  # Get response from rappi
  response = requests.post('https://services.rappi.com.br/api/cpgs/global-search/search', headers=headers, data=data)
  # Read json data from the response
  response = json.loads(response.text)
  # Get column which has the products info
  rappi_data = pd.json_normalize(response).products

  # Getting all products for each store found in the query
  df = pd.DataFrame()
  for k in rappi_data.index:
    df = df.append(pd.json_normalize(rappi_data[k]),ignore_index=True)
    
  # Removing products out of stock
  df.query("real_price < 9999", inplace=True)

  return df

# Setting data and getting results

In [16]:
# Set the keyword between '"keyword"'
keyword = '"Guarana"'

# Set the address to query the stores
address = 'São Paulo'

# Get stores for an address
stores = get_stores(address)

# Web scraping for the address and keyword
rappi_df = rappi(stores['store_id'],keyword)
rappi_df

,id,description,discount_tag_color,min_quantity_in_grams,pum,sale_type,real_price,in_stock,type,unit_type,presentation,price,product_id,store_name,trademark,has_toppings,step_quantity_in_grams,store_id,image,has_antismoking,quantity,is_discontinued,store_type,label,discount_type,has_pum,is_available,balance_price,discount_earnings,real_balance_price,name,max_quantity_in_grams,requires_medical_prescription,age_restriction,popularity,real_trend_pos,corridor_popularity,raw_pop,discount,discount_pay_products,have_discount,restriction_tag
0,900020400_2092884702,REFRIG GUARANA ANTARCTICA 1500ML,,0,2.25,U,4.49,True,,l,1.5 L,3.99,2092884702,"Carrefour, Hiper Cambuci Lion (SPC)",Guaraná Antarctica,False,0,900020400,2093598021-1608128037642.png,False,2,False,carrefour,,,True,True,3.99,,4.49,Guaraná Antarctica Refrigerante 1 5 L Pet,0,False,False,0.000000,0,0,0,0.11,,True,none
1,900020400_2092884707,REFRIG GUARANA ANTARCTICA PET 2L,,0,2.95,U,5.89,True,,l,2 L,5.89,2092884707,"Carrefour, Hiper Cambuci Lion (SPC)",Guaraná Antarctica,False,0,900020400,2098176844-1621366036128.png,False,2,False,carrefour,,,True,True,5.89,,5.89,Guaraná Antarctica Refrigerante,0,False,False,2.300472,0,0,0,0.00,,False,none
2,900020400_2092884698,REFRIG GUARANA ANTARTICA ZERO 1500ML,,0,2.25,U,4.49,True,,l,1.5 L,3.99,2092884698,"Carrefour, Hiper Cambuci Lion (SPC)",Guaraná Antarctica,False,0,900020400,2092913807-1608229303393.png,False,2,False,carrefour,,,True,True,3.99,,4.49,Guaraná Antarctica Refrigerante Zero,0,False,False,0.000000,0,0,0,0.11,,True,none
3,900020400_2093183129,REFRIG GUARANA ANTARTICA PET 1L,,0,,U,3.19,True,,l,1 L,2.99,2093183129,"Carrefour, Hiper Cambuci Lion (SPC)",Guaraná Antarctica,False,0,900020400,2092330755-1608229268284.png,False,1,False,carrefour,,,False,True,2.99,,3.19,Guaraná Antarctica Refrigerante Antartica Pet,0,False,False,0.000000,0,0,0,0.06,,True,none
4,900020400_2092884713,REFRIG GUARANA ANTARCTICA 600ML,,0,0.01,U,3.19,True,,ml,600 mL,3.19,2092884713,"Carrefour, Hiper Cambuci Lion (SPC)",Guaraná Antarctica,False,0,900020400,2098184541-1619808968305.png,False,600,False,carrefour,,,True,True,3.19,,3.19,Guaraná Antarctica Refrigerante Pet,0,False,False,0.000000,0,0,0,0.00,,False,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,900003788_2093768136,Guaraná Cruzeiro Refrigerante Organico 270 mL,,0,0.02,U,5.69,True,,ml,270 mL,5.69,2093768136,"Emporium SP, Afonso Braz",Guaraná Cruzeiro,False,0,900003788,2093768136-1553279009.png,False,270,False,super,,,True,True,5.69,,5.69,Guaraná Cruzeiro Refrigerante Organico,0,False,False,0.000000,0,0,0,0.00,,False,none
297,900003788_2089917236,Refrigerante Guaraná Antarctica Zero 2 L,,0,3.5,U,6.99,True,,l,2 L,6.99,2089917236,"Emporium SP, Afonso Braz",Guaraná Antarctica,False,0,900003788,2089917236-1608229240786.png,False,2,False,super,,,True,True,6.99,,6.99,Guaraná Antarctica Zero 2 L,0,False,False,2.300424,0,0,0,0.00,,False,none
298,900003788_2089917233,Refrigerante brasileiro com sabor único e natu...,,0,0.01,U,2.49,True,,ml,350 mL,2.49,2089917233,"Emporium SP, Afonso Braz",Guaraná Antarctica,False,0,900003788,2098190469-1619809019805.png,False,350,False,super,,,True,True,2.49,,2.49,Guaraná Antarctica Refrigerante Zero,0,False,False,2.301234,0,0,0,0.00,,False,none
299,900003788_2089917234,REFRIG GUAR DIEANTARCTICA600ML,,0,0.01,U,3.19,True,,ml,600 mL,3.19,2089917234,"Emporium SP, Afonso Braz",Guaraná Antarctica,False,0,900003788,2098178624-1619808938008.png,False,600,False,super,,,True,True,3.19,,3.19,Guaraná Antarctica Refrigerante Zero,0,False,False,0.000000,0,0,0,0.00,,False,none
